In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from fbprophet import Prophet

pd.options.display.max_columns = 99
plt.rcParams['figure.figsize'] = (16, 9)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('../input/demand-forecasting-kernels-only/train.csv', parse_dates=['date'], index_col=['date'])
df_test = pd.read_csv('../input/demand-forecasting-kernels-only/test.csv', parse_dates=['date'], index_col=['date'])
df_train.shape, df_test.shape

In [ ]:
df_train.head()

In [ ]:
proph_results = df_test.reset_index()
proph_results['sales'] = 0

In [ ]:
proph_results.head()

In [ ]:
a=df_train.loc[(df_train['store'] == 1) & (df_train['item'] == 1)].reset_index()

In [ ]:
a.head()

In [ ]:
a=df_train.loc[(df_train['store'] == 1) & (df_train['item'] == 1)].reset_index()

In [ ]:
tic = time.time()

for s in proph_results['store'].unique():
    for i in proph_results['item'].unique():
        proph_train = df_train.loc[(df_train['store'] == s) & (df_train['item'] == i)].reset_index()
        proph_train.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)
        
        m = Prophet()
        m.fit(proph_train[['ds', 'y']])
        future = m.make_future_dataframe(periods=len(df_test.index.unique()), include_history=False)
        fcst = m.predict(future)
        
        proph_results.loc[(proph_results['store'] == s) & (proph_results['item'] == i), 'sales'] = fcst['yhat'].values
        
        toc = time.time()
        if i % 10 == 0:
            print("Completed store {} item {}. Cumulative time: {:.1f}s".format(s, i, toc-tic))

In [ ]:
proph_results.drop(['date', 'store', 'item'], axis=1, inplace=True)
proph_results.head()

In [ ]:
proph_results.tail()